In [21]:
import os
import sys
import argparse
import logging
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision.utils import save_image
from PIL import Image, ImageFile
from net import Net
from utils import DEVICE, train_transform, test_transform, FlatFolderDataset, InfiniteSamplerWrapper, plot_grad_flow, adjust_learning_rate

Image.MAX_IMAGE_PIXELS = None  
ImageFile.LOAD_TRUNCATED_IMAGES = True

DEVICE = 'cpu'
# load pretrained checkpoints for the model
os.system("gdown https://drive.google.com/uc?id=1rPB_qnelVVSad6CtadmhRFi0PMI_RKdy")
os.mkdir("checkpoints/")

os.system("unzip -j original_PAMA.zip -d checkpoints/")

Downloading...
From: https://drive.google.com/uc?id=1rPB_qnelVVSad6CtadmhRFi0PMI_RKdy
To: /workspaces/PAMA/original_PAMA.zip
100%|██████████| 158M/158M [00:06<00:00, 24.9MB/s] 


Archive:  original_PAMA.zip
  inflating: checkpoints/.DS_Store   
  inflating: checkpoints/PAMA1.pth   
  inflating: checkpoints/PAMA3.pth   
  inflating: checkpoints/PAMA2.pth   
  inflating: checkpoints/decoder.pth  
  inflating: checkpoints/encoder.pth  


0

In [22]:
def eval(args):
    mes = "current pid: " + str(os.getpid())
    print(mes)
    logging.info(mes)
    model = Net(args)
    model.eval()
    model = model.to(DEVICE)
    
    tf = test_transform()
    if args.run_folder == True:
        content_dir = args.content 
        style_dir = args.style
        for content in os.listdir(content_dir):
            for style in os.listdir(style_dir):
                name_c = content_dir + content
                name_s = style_dir + style
                Ic = tf(Image.open(name_c)).to(DEVICE)
                Is = tf(Image.open(name_s)).to(DEVICE)
                Ic = Ic.unsqueeze(dim=0)
                Is = Is.unsqueeze(dim=0)
                with torch.no_grad():
                    Ics = model(Ic, Is)

                name_cs = "ics/" + os.path.splitext(content)[0]+"--"+style 
                save_image(Ics[0], name_cs)
    else:
        Ic = tf(Image.open(args.content)).to(DEVICE)
        Is = tf(Image.open(args.style)).to(DEVICE)

        Ic = Ic.unsqueeze(dim=0)
        Is = Is.unsqueeze(dim=0)
        
        with torch.no_grad():
            Ics = model(Ic, Is)

        name_cs = "ics.jpg"
        save_image(Ics[0], name_cs)

In [25]:
args = type('obj', (object,), {})
workspace = './'
args.content_dir = workspace + 'content/'
args.style_dir = workspace + 'style/'
args.content = args.content_dir + '1c.png'
args.style = args.style_dir + '1s.png'
args.training = False
args.run_folder = False
args.alpha = 1.0
args.pretrained = True
args.requires_grad = False

eval(args)

current pid: 19560


RuntimeError: Given groups=1, weight of size [3, 3, 1, 1], expected input[1, 4, 513, 512] to have 3 channels, but got 4 channels instead